In [1]:
import pandas as pd
from pyspark.sql.functions import col, explode, array, struct, lit

df = pd.DataFrame({
                   'monday':['apple','apple','banana', 'coconut'],
                   'tuesday':[1, 2, 4, None],
                   'extra':[10, 11, 12, 13],
})

df = spark.createDataFrame(df)

In [2]:
def to_melt(df, id_vars, value_vars):
    ''' 
    Similar to pandas melt, explodes columns into rows.
    
    :param id_vars:  List of columns to keep.
    :param value_vars:  List of column to explode.
    :return:  dataframe
    '''
    
    # Create and explode an array of (column_name, column_value) structs
    kvs = explode(array([
      struct(lit(c).alias('variables'), col(c).alias('values')) for c in value_vars
    ])).alias("kvs")

    return df.select(id_vars + [kvs]).select(id_vars + ['kvs.variables', 'kvs.values'])
  
test = to_melt(df, ['extra'], ['monday', 'tuesday'])

In [3]:
display(test)

extra,variables,values
10,monday,apple
10,tuesday,1.0
11,monday,apple
11,tuesday,2.0
12,monday,banana
12,tuesday,4.0
13,monday,coconut
13,tuesday,null
